In [74]:
from imutils import contours 
from skimage import measure  
import numpy as np          
import argparse
import imutils
import cv2
import os
import matplotlib.pyplot as plt
import time

In [102]:
cap = cv2.VideoCapture('/Users/jessie/TUDelft/Q5/VLC&S/eval_04.MOV')

# get frame rate
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('eval_04_time.MOV', fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

output_directory = '/Users/jessie/TUDelft/Q5/VLC&S/frames_time_eval_04/'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # get current timestamp
    timestamp_ms = int(time.time() * 1000)

    # set timestamp as file name
    frame_filename = os.path.join(output_directory, f'{timestamp_ms}.jpg')

    # add tiemstamp to frames
    cv2.putText(frame, f"Timestamp: {timestamp_ms} ms", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    out.write(frame)

    cv2.imwrite(frame_filename, frame)

    cv2.imshow('Frame with Timestamp', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_number += 1

cap.release()
out.release()
cv2.destroyAllWindows()

print(f'{frame_number} frames have been stored in {output_directory}')


513 frames have been stored in /Users/jessie/TUDelft/Q5/VLC&S/frames_time_eval_04/


In [105]:
folder_path = '/Users/jessie/TUDelft/Q5/VLC&S/frames_time_eval_04/'

# get all file in folder
file_list = sorted(os.listdir(folder_path))

# filter images
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
image_files = [f for f in file_list if os.path.splitext(f)[1].lower() in image_extensions]

# intialize result array
raw_frames_data = [[]]

# traverse all the images
for image_file in image_files:
    
    image_path = os.path.join(folder_path, image_file)
    
    timestamp = int(os.path.splitext(image_file)[0])
    
    # read images
    image = cv2.imread(image_path)
    ## convert BGR to greyscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ## blur the image with a gussian filter to remove any noise, and also to soften the image
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)
    
    # Thresholds, find lightest spots
    thresh = cv2.threshold(blurred, 150 , 255 , cv2.THRESH_BINARY)[1]
    # Remove blobs of noise
    ## perform a series of erosions and dilations to remove
    ## any small blobs of noise from the thresholded image
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=4)

    # Extract contiguous bright islands
# -------------------------------------!!!!!!!!!!!!!!!!!!!!!!!!!-------------------------------------
    # When changing distance,  MIN_ALLOWED_PIXEL_IN_REGIO need to be changed!!!
    # 75cm: 100,    50cm: 400,    20cm: 1000
    MIN_ALLOWED_PIXEL_IN_REGION = 1000
    MAX_GREY_VALUE = 255
    
    ## find out regions and label them
    labels = measure.label(thresh, connectivity=2, background=0)
    
    ## The list holding all contiguous masks that will be found
    individual_masks = []
    
    ## loop over the unique components
    for label in np.unique(labels):
        # if this is the background (dark) label, ignore it
        if label == 0:
            continue
            
        # otherwise, construct the label mask and count the number of pixels 
        labelMask = np.zeros(thresh.shape, dtype="uint8")
        labelMask[labels == label] = MAX_GREY_VALUE
        numPixels = cv2.countNonZero(labelMask)
    
        
        # this condition filters "very" small regions
        if numPixels > MIN_ALLOWED_PIXEL_IN_REGION:
            # plt.figure()        
            individual_masks.append(labelMask)

    # Find out the similarity of the regions to a circle

    ## make a copy of the individual_mask to avoid them from being overwritten
    individual_masks_cp = [mask.copy() for mask in individual_masks]

    # 1 = light is on, 0 = light is off
    light = 0
    
    for idx , current_mask in enumerate(individual_masks_cp):
        cnts = cv2.findContours(current_mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        #cnts = contours.sort_contours(cnts)[0]
    
        ## create a mask to hold the values of the enclosing circle, we also fill inside the cirlce. 
        filled_circle = np.zeros(current_mask.shape)
    
        ## find the coordinates of the center and the radius of the circle
        ((cX, cY), radius) = cv2.minEnclosingCircle(cnts[0])
        
        ## fill the mask values surrounded by the circle
        cv2.circle(filled_circle, (int(cX), int(cY)), int(radius),(255, 255, 255), -1)
        
        ## count the number of pixels in the circle
        circle_area = cv2.countNonZero(filled_circle)
        
        ## count the number of non-zero pixels in the original region
        mask_non_zero_area = cv2.countNonZero(individual_masks[idx])
# ------------------------------------------!!!!!!!!!!!!!!!!!--------------------------------------------
        # when changing LED frequency, this need to be changed
        # 1/3 rate: 0.9,    1/2 rate: 0.7
        # when changing ambient ligth, this need to be changed
        # lighter --> lower
        if mask_non_zero_area / circle_area >= 0.60:
            light = 1

    data_to_add = [light, timestamp]
    raw_frames_data.append(data_to_add)
    print(data_to_add)  
print("Done")

[0, 1697534334788]
[0, 1697534334851]
[0, 1697534334891]
[0, 1697534334928]
[0, 1697534334964]
[0, 1697534335001]
[0, 1697534335036]
[0, 1697534335072]
[0, 1697534335109]
[0, 1697534335145]
[0, 1697534335180]
[0, 1697534335216]
[0, 1697534335251]
[0, 1697534335288]
[0, 1697534335325]
[0, 1697534335360]
[0, 1697534335396]
[0, 1697534335432]
[0, 1697534335468]
[0, 1697534335504]
[0, 1697534335539]
[0, 1697534335575]
[0, 1697534335611]
[0, 1697534335647]
[0, 1697534335683]
[0, 1697534335719]
[0, 1697534335755]
[0, 1697534335790]
[0, 1697534335827]
[0, 1697534335862]
[0, 1697534335898]
[0, 1697534335935]
[0, 1697534335971]
[0, 1697534336007]
[0, 1697534336043]
[0, 1697534336078]
[0, 1697534336117]
[0, 1697534336152]
[0, 1697534336188]
[0, 1697534336226]
[0, 1697534336262]
[0, 1697534336298]
[0, 1697534336335]
[0, 1697534336370]
[0, 1697534336406]
[0, 1697534336442]
[0, 1697534336478]
[0, 1697534336516]
[0, 1697534336552]
[0, 1697534336589]
[0, 1697534336625]
[0, 1697534336661]
[0, 16975343

In [106]:
for i, item in enumerate(raw_frames_data):
    
    if len(item) != 2:
        continue
    value, timestamp = item
    
    if value == 1:
        break
    # i if the first 1 appears
    if i > 0:
        frames_data = raw_frames_data[(i+1):]
    else:
        frames_data = None

print(frames_data)

[[1, 1697534338081], [1, 1697534338119], [1, 1697534338155], [0, 1697534338190], [0, 1697534338226], [0, 1697534338261], [0, 1697534338297], [0, 1697534338333], [0, 1697534338369], [1, 1697534338404], [1, 1697534338440], [1, 1697534338475], [1, 1697534338511], [1, 1697534338547], [1, 1697534338582], [0, 1697534338618], [0, 1697534338654], [0, 1697534338689], [0, 1697534338725], [0, 1697534338762], [0, 1697534338798], [1, 1697534338833], [1, 1697534338869], [1, 1697534338905], [1, 1697534338941], [1, 1697534338977], [1, 1697534339013], [0, 1697534339049], [0, 1697534339085], [0, 1697534339120], [0, 1697534339156], [0, 1697534339191], [0, 1697534339227], [1, 1697534339262], [1, 1697534339298], [1, 1697534339334], [1, 1697534339369], [1, 1697534339404], [1, 1697534339438], [0, 1697534339474], [0, 1697534339509], [0, 1697534339545], [0, 1697534339580], [0, 1697534339616], [0, 1697534339651], [1, 1697534339686], [1, 1697534339722], [1, 1697534339758], [1, 1697534339793], [1, 1697534339828],

In [107]:
Manchester_codes = []

# set threshold
# ----------------------------!!!!!!!!!!!!!!!!!---------------------------
# When changing LED frequency, this need to be changed
# 1/3 rate: 170,     1/2 rate: 112

threshold = 160

current_group = []

for item in frames_data:
    
    if len(item) != 2:
        continue 
        
    value, timestamp = item
    
    if not current_group:
        # if current_group is empty, means a new one start
        current_group.append(value)
    elif value != current_group[-1]:
        # if current value is different from last one in current_group, this current_group end
        length = len(current_group)
        time_interval = timestamp - frames_data[frames_data.index([value, timestamp]) - length][1]
        
        if time_interval >= threshold:
            if current_group[-1] == 1:
                Manchester_codes.extend([1, 1])
            elif current_group[-1] == 0:
                Manchester_codes.extend([0, 0])
        else:
            if current_group[-1] == 1:
                Manchester_codes.append(1)
            elif current_group[-1] == 0:
                Manchester_codes.append(0)
        current_group = [value]
    else:
        # current value = value in current_group
        current_group.append(value)

if current_group:
    # last group of value
    last_timestamp = frames_data[-1][1]
    length = len(current_group)
    time_interval = last_timestamp - frames_data[frames_data.index([current_group[0], last_timestamp]) - length][1]
    if time_interval >= threshold:
        if current_group[-1] == 1:
            Manchester_codes.extend([1, 1])
        elif current_group[-1] == 0:
            Manchester_codes.extend([0, 0])
    else:
        if current_group[-1] == 1:
            Manchester_codes.append(1)
        elif current_group[-1] == 0:
            Manchester_codes.append(0)

print(Manchester_codes)
print(len(Manchester_codes))

[1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0]
130


In [108]:
# convert Manchester code to binary data
binary_data = []
i = 1
while i < len(Manchester_codes):
    if Manchester_codes[i-1] == 1 and Manchester_codes[i] == 0:
        binary_data.append(1)
        i += 2
    elif Manchester_codes[i-1] == 0 and Manchester_codes[i] == 1:
        binary_data.append(0)
        i += 2
    else:
        print("Invalid Manchester encoding sequence.")
        break
print("Decoded Binary Data:", binary_data)
print(len(binary_data))

Invalid Manchester encoding sequence.
Decoded Binary Data: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0]
64


In [109]:
# check preamble
preamble = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
if binary_data[:len(preamble)] == preamble:
    print("Preamble is OK")
else:
    print("Preamble is Wrong")
    
# check message length
message_length = int("".join(map(str, binary_data[len(preamble):len(preamble) + 16])), 2)
if (message_length + 5 + 2) * 8 == len(binary_data):
    print("Message Length is Right")
else:
    print("Message Length is Wrong")

Preamble is OK
Message Length is Right


In [110]:
# convert payload to readable message
start_index = 5 * 8
end_index = -16
payload = ''.join(map(str, binary_data[start_index:end_index]))
# print(payload)
bytes_payload = [payload[i:i+8] for i in range(0, len(payload), 8)]
# print(bytes_payload)

ascii_string = "".join([chr(int(byte, 2)) for byte in bytes_payload])
# decimal_payload = int(payload, 2)
# ascii_char = chr(decimal_value)

print("Readable Message:", ascii_string)

Readable Message: a
